# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

In [177]:
import pandas as pd
import numpy as np
import math

## Загрузка и предобработка данных

Загрузим данные и проведем предобраотку данных как на семинаре.

In [178]:
!wget -q -N https://www.dropbox.com/s/btgtc07robwlrfr/articles.zip?dl=0 -O articles.zip
!unzip -o -q articles.zip

In [179]:
!rm articles.zip

In [180]:
articles_df = pd.read_csv('/content/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [181]:
interactions_df = pd.read_csv('/content/users_interactions.csv')
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [182]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [183]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [184]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())
print('# users:', len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions:',len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [185]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [186]:
print('# interactions before:', interactions_df.shape)
print('# interactions after:', interactions_from_selected_users_df.shape)

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [187]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [188]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [189]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), 'true_test'] = [
    list() for x in range(len(interactions.loc[pd.isnull(interactions.true_test), 'true_test']))]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


## Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [190]:
!pip install lightfm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## Задание 1. (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [191]:
# Ваш код здесь

from lightfm.data import Dataset

data = Dataset()
data.fit(interactions_full_df.personId.unique(), interactions_full_df.contentId.unique())
interactions_matrix_train, weights_matrix_train = data.build_interactions([tuple(i) for i in interactions_train_df.drop('last_timestamp', axis = 1).values])
interactions_matrix_test, weights_matrix_test = data.build_interactions([tuple(i) for i in interactions_test_df.drop('last_timestamp', axis = 1).values])

In [192]:
interactions_matrix_train

<1140x2984 sparse matrix of type '<class 'numpy.int32'>'
	with 29329 stored elements in COOrdinate format>

In [193]:
interactions_matrix_test

<1140x2984 sparse matrix of type '<class 'numpy.int32'>'
	with 9777 stored elements in COOrdinate format>

In [194]:
 weights_matrix_test.toarray()[1][:100]

array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 2.7004397, 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 2.7004397, 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 1.       , 0.       ,
       1.       , 2.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 1.       , 2.807355 , 0.       ,
       0.       , 0.       , 0.       , 0.       , 

## Задание 2. (1 балл)

Обучите модель LightFM с `loss='warp'` и посчитайте *precision@10* на тесте.

In [195]:
# Ваш код здесь
from lightfm import LightFM

In [196]:
model = LightFM(loss='warp')
%time model.fit(weights_matrix_train)

CPU times: user 51.4 ms, sys: 3.5 ms, total: 54.9 ms
Wall time: 62.8 ms


In [197]:
from lightfm.evaluation import precision_at_k

In [198]:
precision_at_k(model, weights_matrix_test, k=10).mean()

0.0024439918

## Задание 3. (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss='warp'` и посчитайте precision@10 на тесте.

In [199]:
print(len(articles_df['text'].unique())) # для некоторых статей нет текста
print(len(articles_df['contentId'].unique()))
print(len(interactions_full_df[interactions_full_df['eventStrength'].notna()]['contentId'].unique())) # для некоторых статей нет весов взаимодействий, т.к. мы отфильтровали

3013
3047
2984


In [200]:
merged_df = pd.merge(left=interactions_full_df, right=articles_df[['contentId', 'text']], left_on='contentId', right_on='contentId')

In [201]:
# Ваш код здесь

from sklearn.feature_extraction.text import TfidfVectorizer

#articles_df_sorted = articles_df.sort_values(by=['contentId'])

subset = merged_df.drop_duplicates(subset=['contentId'])

corpus = list(subset['text'])

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())

print(X.shape)

feat = X

['00', '000', '0000', '000000', '000000000001', '0000000000400848', '000001', '000001000001', '0001', '000707', '000831', '000km', '000x256x256x3', '001', '001052', '001677', '0018', '001g', '002', '003265628676327108248', '0036', '004', '00455455mb17', '005', '007', '007260', '007453', '008412', '009', '00am', '00ff00', '00pm', '00z', '01', '010', '010477', '011787978z', '011808021z', '011814273z', '011819429z', '011824960z', '011834803z', '011839910z', '011845543z', '011850557z', '011855603z', '011860984z', '011865623z', '011871858z', '011877088z', '012', '014', '014938', '015', '0157', '017', '019', '01m', '01nesse', '02', '020728', '022', '022408', '023', '0235', '025', '026', '027', '029', '03', '031', '034', '038733', '03zloezm19bjml14ump0r7lkgxegngbspzxxmtk19hmznwmmblhajf1vaoqqarq2txj7gnlznl8rpkqfszsk40n', '04', '041083333333333326', '043', '045', '0459', '048', '05', '0500', '0502285', '0511', '052', '053', '058020770920711007', '05min', '05t17', '05u', '06', '060', '060ms', '0

In [202]:
print(feat.toarray()[1][:100])
feat
print(len(merged_df['contentId'].unique()))

[0.         0.01411053 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
2976


In [203]:
split_ts = 1475519530

train_merged_df= merged_df.loc[merged_df.last_timestamp < split_ts].copy()
test_merged_df= merged_df.loc[merged_df.last_timestamp >= split_ts].copy()

print('# interactions on Train set: %d' % len(train_merged_df))
print('# interactions on Test set: %d' % len(test_merged_df))

train_merged_df

# interactions on Train set: 29302
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp,text
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911,A AXA se manteve na liderança do ranking de ma...
1,-1352542225971050638,-5065077552540450930,1.000000,1470415770,A AXA se manteve na liderança do ranking de ma...
2,-1393866732742189886,-5065077552540450930,2.321928,1470353620,A AXA se manteve na liderança do ranking de ma...
3,-2826197582088460753,-5065077552540450930,1.000000,1470398034,A AXA se manteve na liderança do ranking de ma...
4,-510621402112686165,-5065077552540450930,1.000000,1470685665,A AXA se manteve na liderança do ranking de ma...
...,...,...,...,...,...
39073,8754745127801506293,-3113913063173722290,1.000000,1464356403,Maurice Ashley (Foto: Reprodução/ Facebook) Ma...
39074,881856221521045800,4106497696154898573,1.000000,1468861886,"Lesson 1: Swift Basics In this lesson, you'll ..."
39075,8855523843512271162,-8464215556093549753,1.584963,1467206922,Join Women's CodeSprint to prove your coding s...
39077,8968131284214320024,5518462222339671372,1.000000,1471844669,"It won't be easy, but shifting to a productivi..."


In [204]:
data = Dataset()
data.fit(merged_df.personId.unique(), merged_df.contentId.unique())
interactions_matrix_train, weights_matrix_train = data.build_interactions([tuple(i) for i in train_merged_df.drop(['last_timestamp', 'text'], axis = 1).values])
interactions_matrix_test, weights_matrix_test = data.build_interactions([tuple(i) for i in test_merged_df.drop(['last_timestamp', 'text'], axis = 1).values])

In [205]:
%time model.fit(weights_matrix_train, item_features=feat)

CPU times: user 3.97 s, sys: 3.78 ms, total: 3.97 s
Wall time: 3.97 s


In [206]:
precision_at_k(model, weights_matrix_test, k=10, item_features=feat).mean()

0.004480652

## Задание 4. (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [207]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from string import punctuation
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [221]:
import spacy
nlp = spacy.load('en')

In [226]:
# Ваш код здесь

# articles_df['lang'].unique() array(['en', 'pt', 'es', 'la', 'ja'], dtype=object)
exclude = set(punctuation)
all_stopwords = stopwords.words('english')
all_stopwords.extend(stopwords.words('portuguese'))
all_stopwords.extend(stopwords.words('spanish'))
exclude.update(all_stopwords)

def preprocess_text(text):
    return ' '.join([word for word in word_tokenize(text.lower()) if word not in all_stopwords])
    #return ' '.join(words_lemmata_list)

In [227]:
subset = merged_df.drop_duplicates(subset=['contentId'])

subset['text_new'] = subset['text'].apply(preprocess_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [228]:
subset[:3]

,personId,contentId,eventStrength,last_timestamp,text,text_new
0,-1007001694607905623,-5065077552540450930,1.0,1470395911,A AXA se manteve na liderança do ranking de ma...,axa manteve liderança ranking maiores segurado...
32,-1007001694607905623,-6623581327558800021,1.0,1487240080,"About a decade ago, a handful of Google's most...","decade ago , handful google 's talented engine..."
88,-1007001694607905623,-793729620925729327,1.0,1472834892,"Posted by Sam Thorogood , Developer Programs E...","posted sam thorogood , developer programs engi..."


In [229]:
corpus = list(subset['text_new'])

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())

print(X.shape)

feat = X

['00', '000', '0000', '000000', '000000000001', '0000000000400848', '000001', '000001000001', '0001', '000707', '000831', '000km', '000x256x256x3', '001', '001052', '001677', '0018', '001g', '002', '003265628676327108248', '0036', '004', '00455455mb17', '005', '007', '007260', '007453', '008412', '009', '00am', '00ff00', '00pm', '00z', '01', '010', '010477', '011787978z', '011808021z', '011814273z', '011819429z', '011824960z', '011834803z', '011839910z', '011845543z', '011850557z', '011855603z', '011860984z', '011865623z', '011871858z', '011877088z', '012', '014', '014938', '015', '0157', '017', '019', '01m', '01nesse', '02', '020728', '022', '022408', '023', '0235', '025', '026', '027', '029', '03', '031', '034', '038733', '03zloezm19bjml14ump0r7lkgxegngbspzxxmtk19hmznwmmblhajf1vaoqqarq2txj7gnlznl8rpkqfszsk40n', '04', '041083333333333326', '043', '045', '0459', '048', '05', '0500', '0502285', '0511', '052', '053', '058020770920711007', '05min', '05t17', '05u', '06', '060', '060ms', '0

In [ ]:
split_ts = 1475519530

train_merged_df= merged_df.loc[merged_df.last_timestamp < split_ts].copy()
test_merged_df= merged_df.loc[merged_df.last_timestamp >= split_ts].copy()

data = Dataset()
data.fit(merged_df.personId.unique(), merged_df.contentId.unique())
interactions_matrix_train, weights_matrix_train = data.build_interactions([tuple(i) for i in train_merged_df.drop(['last_timestamp', 'text'], axis = 1).values])
interactions_matrix_test, weights_matrix_test = data.build_interactions([tuple(i) for i in test_merged_df.drop(['last_timestamp', 'text'], axis = 1).values])


In [237]:
model = LightFM(loss='warp')
%time model.fit(weights_matrix_train, item_features=feat)
precision_at_k(model, weights_matrix_test, k=10, item_features=feat).mean()

CPU times: user 3.29 s, sys: 10 ms, total: 3.3 s
Wall time: 3.28 s


0.0042769862

Улучшилось ли качество предсказания?

Spacy не обучился, поэтому использовала tokenizer. Качество не улучшилось.

## Задание 5. (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [238]:
# Ваш код здесь

model = LightFM(loss='warp', no_components=30, learning_schedule = 'adagrad', learning_rate = 0.01, max_sampled=20)

%time model.fit(weights_matrix_train, item_features=feat, epochs=30, verbose=True)
precision_at_k(model, weights_matrix_test, k=10, item_features=feat).mean() 



Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch:   3%|▎         | 1/30 [00:09<04:40,  9.68s/it]

Epoch:   7%|▋         | 2/30 [00:19<04:33,  9.76s/it]

Epoch:  10%|█         | 3/30 [00:30<04:31, 10.05s/it]

Epoch:  13%|█▎        | 4/30 [00:41<04:28, 10.34s/it]

Epoch:  17%|█▋        | 5/30 [00:52<04:25, 10.63s/it]

Epoch:  20%|██        | 6/30 [01:04<04:20, 10.87s/it]

Epoch:  23%|██▎       | 7/30 [01:15<04:12, 10.99s/it]

Epoch:  27%|██▋       | 8/30 [01:26<04:05, 11.16s/it]

Epoch:  30%|███       | 9/30 [01:37<03:51, 11.04s/it]

Epoch:  33%|███▎      | 10/30 [01:48<03:40, 11.02s/it]

Epoch:  37%|███▋      | 11/30 [01:59<03:28, 10.96s/it]

Epoch:  40%|████      | 12/30 [02:10<03:17, 10.96s/it]

Epoch:  43%|████▎     | 13/30 [02:21<03:06, 10.96s/it]

Epoch:  47%|████▋     | 14/30 [02:32<02:55, 10.98s/it]

Epoch:  50%|█████     | 15/30 [02:43<02:45, 11.06s/it]

Epoch:  53%|█████▎    | 16/30 [02:55<02:36, 11.20s/it]

Epoch:  57%|█████▋    | 17/30 [03:07<02:28, 11.40s/it]

Epoch:  

CPU times: user 5min 14s, sys: 280 ms, total: 5min 14s
Wall time: 5min 13s


0.0052953153

## Бонусное задание. (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

In [ ]:
# Ваш код здесь